In [13]:
import os
import sys
import json
import logging
from typing import Dict

In [15]:
sys.path.append(os.path.dirname(os.getcwd()))
from configs import BASE_MAPPING_PATH, QANDA_FILE_READER_PATH
from client import (
    OsMlClientWrapper,
    get_client,
)
from data_process import QAndAFileReader
from mapping import get_base_mapping
from main import load_category

ImportError: cannot import name 'QANDA_FILE_READER_PATH' from 'configs' (/home/ec2-user/Code/opensearch-ml-quickstart/configs/__init__.py)

In [ ]:
def load_dataset_lexical(
    client: OsMlClientWrapper,
    pqa_reader: QAndAFileReader,
    config: Dict[str, str],
    delete_existing: bool,
    index_name: str,
):
    if delete_existing:
        logging.info(f"Deleting existing index {index_name}")
        client.delete_then_create_index(
            index_name=config["index_name"], settings=config["index_settings"]
        )

    logging.info("Setting up without KNN")
    client.setup_without_kNN(
        index_name=config["index_name"],
        index_settings=config["index_settings"],
    )

    for category in config["categories"]:
        load_category(
            client=client.os_client,
            pqa_reader=pqa_reader,
            category=category,
            config=config,
        )

In [ ]:
host_type = "aos"
index_name = "lexical_search"
dataset_path = QANDA_FILE_READER_PATH
number_of_docs = 500
client = OsMlClientWrapper(get_client(host_type))

pqa_reader = QAndAFileReader(
    directory=dataset_path, max_number_of_docs=number_of_docs
)

categories = ["sheet and pillowcase sets"]

config = {
    "categories": categories,
    "index_name": index_name,
    "index_settings": get_base_mapping(BASE_MAPPING_PATH),
}

logging.info(f"Config:\n {json.dumps(config, indent=4)}")

load_dataset_lexical(
    client,
    pqa_reader,
    config,
    delete_existing=True,
    index_name=index_name,
)

In [ ]:
query_text = "I wish to sleep well"
search_query = {
    "_source": {"include": "chunk"},
    "query": {"match": {"chunk": query_text}},
}

In [ ]:

search_results = client.os_client.search(index=index_name, body=search_query)
hits = search_results["hits"]["hits"]
hits = [hit["_source"]["chunk"] for hit in hits]
hits = list(set(hits))
for i, hit in enumerate(hits):
    print(f"{i + 1}th search result:\n {hit}")

In [ ]:
index_name = "dense_exact_search"
search_query = {
    "_source": {"include": "chunk"},
    "query": {
        "neural": {
            "chunk_embedding": {
                "query_text": query_text,
                "model_id": "twKE_pUBdIibWvQqbhrY",
            }
        }
    },
}

In [ ]:
search_results = client.os_client.search(index=index_name, body=search_query)
hits = search_results["hits"]["hits"]
hits = [hit["_source"]["chunk"] for hit in hits]
hits = list(set(hits))
for i, hit in enumerate(hits):
    print(f"{i + 1}th search result:\n {hit}")